# PythonでBlockchainの実装
- [Bitcoin論文](https://bitcoin.org/bitcoin.pdf)をもとにblockchainの実装をする
    - 天下り的にならないよう，徐々にシステムをパワーアップさせていく方針で作る
- 目標はシンプルなコンセンサスアルゴリズムの実装まで行い，ノード同士が競ってマイニングする様子を眺めること
    - 完成形はこんな感じ
    
    <img src="./demo.gif" width=50%>

# 信用のある第三者を介する取引
- まずは第三者を介した取引をシミュレーションしてみる
- 第三者は取引の公正さのすべてを請け負う

## 取引はどのように行うか? - Transactionの実装
- 取引記録を発行することで，取引を行う
    - Python3.7から実装された[dataclasses](https://docs.python.org/ja/3/library/dataclasses.html#)を使ってみる

In [1]:
from __future__ import annotations
from dataclasses import dataclass

- 個人の識別を`address`で行う
- 取引額を`value`とする

In [2]:
@dataclass
class Transaction:
    sender_address: int
    receiver_address: int
    value: float 

- senderはTransactionを発行して取引を行う

## Walletの実装
- とりあえず，Walletは適当な1つの`int`をaddressとして持つことにする
- Transactionを発行する機能をもたせる
    - `send` methodで取引記録を作成できる

In [4]:
@dataclass
class Wallet:
    address: int
    def send(self, receiver_address, value):
        return Transaction(self.address, receiver_address, value)

## Ledgerの実装
- transactionは信用のある第三者(Bankと呼ぶことにする)に提出されることで成立する
    - BankはTransactionを元帳(Ledger)で管理する
    - Bankが全てのTransactionを管理し，精査することで不正を防止する
- Ledgerはとりあえず単なるリストにする

In [5]:
Ledger=list

## デモ: シンプルな取引
- AliceとBobが取引する様子をデモする

In [6]:
alice=Wallet(1) # create Alice's wallet
bob=Wallet(2)   # create Bob's wallet
ledger=Ledger() # create Bank's ledger

- Aliceがtransactionを発行してBankに登録することで，取引が成立する

In [7]:
transaction=alice.send(bob.address, 5) # Alice send `5` to Bob
ledger.append(transaction)

- ここで例えば，第三者の信頼度がちょっと落ちたとする
    - 取引内容の改ざんはどのように防げばよいだろうか?

# 電子署名を用いたtransactionの発行
- senderはtransactionに電子署名を行う
    - 今回はRSAを用いることにする
        - BitCoinは[ECDSA](https://en.bitcoin.it/wiki/Transaction#Input)を用いている
    - これによってtransactionの改ざんを検知することができる
- 電子署名の公開鍵はどのように扱う?
    - 公開鍵をWalletのaddressとする!

## Transactionの改良 - 電子署名フィールドの追加
- 電子署名できるようにする
    - `sign`を追加
    - 署名の際，transactionをstrで表すための`str_data` methodも追加
- ついでにデータをjson stringに変換するmethodを追加する
    - これは後々データのやり取りを簡単にするため

In [8]:
import json
import dataclasses as dc

In [9]:
@dataclass
class Transaction:
    sender_address: str
    receiver_address: str
    value: float
    sign: str = None
        
    def str_data(self) -> str:
        # return the str of data without `sign`
        d=dc.asdict(self)
        del d["sign"]
        return json.dumps(d)
    
    def json_dumps(self) -> str:
        return json.dumps(dc.asdict(self))
    @classmethod
    def json_loads(cls, string) -> Transaction:
        return cls(**json.loads(string))

## Walletの改良 - 電子署名機能の追加
- RSAによる電子署名機能を実装する
    - pycryptoを用いる
        - 使い方はこれを参照 ([python \- Signing and verifying data using pycrypto \(RSA\) \- Stack Overflow](https://stackoverflow.com/questions/4232389/signing-and-verifying-data-using-pycrypto-rsa))
- 引き続きWalletはaddressを1つのみ持つことにする
    - 例えばBitCoinでは匿名性の向上のため，一つのWalletは複数のキーペアを持つことができる ([参考](https://en.bitcoin.it/wiki/Transaction))

In [10]:
from Crypto.Signature import PKCS1_v1_5
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
import binascii

- pycryptoのkey instanceをstrに変換するhelper functionを実装する
    - strにしておく理由は後ほどわかる

In [11]:
def decode_key(key):
    if isinstance(key, str):
        return key
    return binascii.hexlify(key.exportKey(format='DER')).decode('ascii')
def encode_key(key):
    if isinstance(key, RSA._RSAobj):
        return key
    return RSA.importKey(binascii.unhexlify(key))

Walletの改良

In [12]:
class Wallet:
    def __init__(self):
        key=RSA.generate(1024)
        self.private_key = decode_key(key)
        self.address = decode_key(key.publickey())
    
    def sign_transaction(self, transaction) -> Transaction:
        # generate signer from self private key
        signer=PKCS1_v1_5.new(encode_key(self.private_key))
        # hash the transaction
        h=SHA.new(transaction.str_data().encode())
        # add signature to the transaction, and return it
        return dc.replace(transaction, sign=signer.sign(h).hex())
    
    def send(self, receiver_address, value) -> Transaction:
        """Generate transaction, and add signature"""
        transaction=Transaction(self.address, receiver_address, value)
        return self.sign_transaction(transaction)

- 秘密鍵`private_key`の追加 
- addressは対応する公開鍵とする
- `sign_transaction`の追加
    - transactionを受け取り，電子署名付きtransactionを返す
- `send`を変更
    - 電子署名付きtransactionを生成できるようになった!

つぎに，transactionの電子署名をチェックするhelper functionを作る
- signatureのpublic keyはtransactionのaddressなので，transactionを見れば電子署名の正当性を判別できる

In [13]:
def verify_transaction(transaction) -> bool:
    if transaction.sign is None:
        return False
    # hash the transaction
    h=SHA.new(transaction.str_data().encode())
    # generate verifier with public key
    verifier=PKCS1_v1_5.new(encode_key(transaction.sender_address))
    # is the signature correct?
    return verifier.verify(h, binascii.unhexlify(transaction.sign))

## デモ: RSAを用いたtransactionの発行とチェック
先ほどと同じようにデモをしてみる

In [14]:
alice=Wallet() # now the address is automatically generated
bob=Wallet()
ledger=Ledger()

- transactionを発行し，電子署名を行い，bankに提出する

In [15]:
transaction=alice.send(bob.address, 5)
# Bank append the transaction to its ledger
ledger.append(transaction)

- transactionの電子署名を確認してみる

In [16]:
verify_transaction(transaction)

True

- データをちょと変えてみると，不正なtransactionになるのがわかる

In [17]:
transaction.value=7
verify_transaction(transaction)

False

- transactionの改竄リスクを低減することができた 
    - しかし依然として，**double-spending problem** を防ぐには信頼のある第三者が必要となる
    - これはデータで通貨を作る際の最も大きな障害だった(データは簡単にコピーできるから)
- すべてのtransactionが公開されていれば，double-spendingは防げるのでは？
    - 公開されていればreceiverも記録をチェックできる

# Timestamp Server によるTransactionの公開
- いくつかのtransactionをまとめて，timestampをつけ，**block**で公開することを考える
    - 信頼のある**timestamp server**がtransactionをまとめてtimestampをつけ，電子署名をして公開する
    - ある特定の時刻にそのtransactionが存在したことが保証される
- さらに，blockには直前のtimestampのhashを含め，**blockchain**を作る
    - blockを改ざんすると，以降のblockは全て不正なものとなる
- これでtransactionをチェックしてdouble-spendingを防ぐ第三者は必要なくなった
    - transactionは公開されているから，誰でも検証することができる

## Blockの実装
- strで表現できるよう，json methodを追加する
- ついでにhash methodも実装する

In [18]:
from typing import Tuple, Sequence
import hashlib

In [59]:
@dataclass
class Block:
    time: float
    transactions: Tuple[Transaction]
    previous_hash: str
    sign: str = None
        
    def json_dumps(self) -> str:
        dct=dc.asdict(self)
        dct["transactions"]=[t.json_dumps() for t in self.transactions]
        return json.dumps(dct)
    @classmethod
    def json_loads(cls, string) -> Block:
        dct=json.loads(string)
        dct["transactions"]=tuple([Transaction.json_loads(t) for t in dct["transactions"]])
        return cls(**dct)
        
    def hash(self) -> str: 
        block_bytes=self.json_dumps().encode()
        return hashlib.sha256(block_bytes).hexdigest()

## Timestamp Serverの実装
- timestamp serverは以下の流れで**blockchain**を作る
    1. transactionのリストを受け取る
    2. 一つ前のblockのhashとtimestampをつけてBlockにまとめる
    3. **電子署名**をする
    4. 公開する

- blockの正当性の保証はtimestamp serverの**電子署名**で行う
    - serverに RSA キーペアをもたせる
- 最初のblockはprevious blockがないため，特別なブロック(**genesis**)を用意する ([参考](https://en.bitcoin.it/wiki/Genesis_block))
- blockchainは今回はただのリストにする

In [60]:
from time import time

In [ ]:
class TimestampServer:
    def __init__(self):
        key=RSA.generate(1024)
        self.public_key=key.publickey()
        self.signer=PKCS1_v1_5.new(key)
        
        genesis=Block(time(), (), "0")
        self.block_chain=[genesis]
        
    def generate_block(self, transactions: Sequence[Transaction]):
        # generate block
        block=Block(time(), tuple(transactions), self.block_chain[-1].hash())
        
        # sign the block
        dct=dc.asdict(block)
        del dct["sign"]
        block.sign=self.signer.sign(SHA.new(json.dumps(dct).encode())).hex()
        
        # publish the block
        self.block_chain.append(block)

## blockchainの正当性確認
- blockchainの正当性を確認するhelper functionを用意する
    1. blockの持つ**previous_hash**は正しいか?
    2. blockに入っているtransactionは正当か?
    3. timestamp serverがつけた電子署名は正当か?

- blockの正当性確認

In [22]:
def verify_block(previous_block, block, timestamp_server_publickey) -> bool:
    is_correct_hash = previous_block.hash() ==  block.previous_hash
    is_correct_transactions = all(map(verify_transaction, block.transactions))
    
    dct=dc.asdict(block)
    del dct["sign"]
    h=SHA.new(json.dumps(dct).encode())
    verifier=PKCS1_v1_5.new(timestamp_server_publickey)
    is_correct_sign=verifier.verify(h, binascii.unhexlify(block.sign))
    return is_correct_hash and is_correct_transactions and is_correct_sign

- blockchainの正当性確認

In [23]:
def verify_blockchain(chain, timestamp_server_publickey):
    for i in range(len(chain)-1):
        if not verify_block(chain[-i-2],chain[-i-1],timestamp_server_publickey):
            return False
    return True

## デモ: Timestamp server による Blockの公開とBlockChainへの登録
- timestamp serverをつくる

In [24]:
timestamp_server=TimestampServer()

- 取引を複数回行う

In [25]:
transactions=[]
transactions.append(alice.send(bob.address, 5))
transactions.append(bob.send(alice.address, 7))

- Timestamp serverによるBlockの生成と公開
    - timestamp_serverのblockchainは**誰でも見ることができる**

In [26]:
timestamp_server.generate_block(transactions)

- 正当性確認

In [27]:
verify_blockchain(timestamp_server.block_chain, timestamp_server.public_key)

True

- これでtransactionのチェックをすべての人が行えるようになった
- しかし，transactionを検証する第三者は必要なくなったが，今度は信頼のあるtimestamp serverが必要になってしまった
- **timestamp serverを第三者に任せない方法がほしい!**

# Proof-of-Workの実装
- これがblockchainの革新的要素
- 先程は"信頼の置ける" timestamp serverの電子署名が，Blockの正当性を保証していた
- では誰でも同じ署名ができれば，誰でもtimestamp serverの仕事ができるのでは?
    - 電子署名でやると，当然ブロックの正当性の証明にはならない..
- **[HashCash](https://en.bitcoin.it/wiki/Hashcash)**を用いる

## Blockにnonceを加える
- これがtimestamp serverの電子署名の代わりとなる
- blockのハッシュ値が適当な条件を満たすように**nonce**を追加する
    - 例えば上4桁が0
    - この制約を**difficulty**と呼ぶ
    - difficultyは適当に調整される. 例えばbitcoinの場合は，blockの追加のintervalが10分程度になるように調整されるようだ
        - [Difficulty \- Bitcoin Wiki](https://en.bitcoin.it/wiki/Difficulty)
    - 今回はdifficultyは固定にする
- nonceを見つける作業が**mine**
    - nonceを見つけるのは難しい(たくさん試して，あたりを引くしかない)
    - これによってblockの生成を非常に高コストにすることができる
    - nonceが正しいことを確認するのは簡単
        - 電子署名と同様, 正当性の確認が簡単に行える
- **mine**は誰でも行える

- Blockにnonceをつける．今回の変更はこれだけ

In [ ]:
@dataclass
class Block:
    time: float
    transactions: Tuple[Transaction]
    previous_hash: str
    nonce: int = None
        
    def json_dumps(self) -> str:
        dct=dc.asdict(self)
        dct["transactions"]=[t.json_dumps() for t in self.transactions]
        return json.dumps(dct)
    
    @classmethod
    def json_loads(cls, string) -> Block:
        dct=json.loads(string)
        dct["transactions"]=tuple([Transaction.json_loads(t) for t in dct["transactions"]])
        return cls(**dct)
        
    def hash(self): 
        block_bytes=self.json_dumps().encode()
        return hashlib.sha256(block_bytes).hexdigest()

## nonce の正当性確認functionを実装

In [63]:
difficulty=3
def valid_proof(block):
    return block.hash()[:difficulty] == "0" * difficulty

## mineの実装
- あたりを引くまで手当たり次第試す

In [64]:
def mine(block):
    nonce=0
    block.nonce=nonce
    while not valid_proof(block):
        nonce += 1
        block.nonce=nonce
    return block

- ついでにverify_blockも改良

In [65]:
def verify_block(previous_block, block) -> bool:
    is_correct_hash = previous_block.hash() ==  block.previous_hash
    is_correct_transactions = all(map(verify_transaction, block.transactions))
    is_correct_proof = valid_proof(block)
    return is_correct_hash and is_correct_transactions and is_correct_proof

- BlockChainもちょっと改良
    - 自身をstrに変換するmethodをつける

In [ ]:
class BlockChain(list):
    def json_dumps(self) -> str:
        return json.dumps([block.json_dumps() for block in self])
    @classmethod
    def json_loads(cls, string) -> BlockChain:
        return cls([Block.json_loads(s) for s in json.loads(string)])

## デモ: tranasctionの発行からminingまで

In [ ]:
genesis=Block(time(), (), "0")
block_chain=BlockChain([genesis])

# 取引
transactions=[]
transactions.append(alice.send(bob.address, 5))
transactions.append(bob.send(alice.address, 5))

- mine

In [34]:
previous_hash=block_chain[-1].hash()
block=mine(Block(time(), tuple(transactions), previous_hash))

In [35]:
block.nonce

2523

- かなり遅い実装をしているので，そこそこ時間がかかるかもしれない
- blockを検証してみる

In [36]:
verify_block(block_chain[-1], block)

True

# NodeとConsensus Algorithm
- 前回までで分散管理の準備は整った
- 具体的にblockchainをつくる**node**を実装する
- nodeやsender, receiverが集まって**network**を形成する
- networkの動きは以下のようになる

1. senderがtransaction生成．なるべく多くのnodeにbroadcastする
2. nodeはtransactionを集めてblockにまとめる
3. mine
4. 他のnodeにマイニングしたblockをbroadcastする
5. blockを受け取ったnodeは，blockの正当性を確認し，double-spendingのチェックをする
6. チェックが通れば，次のblockの作成を始める

- Nodeが複数いて，異なるBlockChainを持つときは，どのように正しさを判断するか？
    - **Consensus Algorithm**
    - 最も長いblockchainを，最も正しいblockchainとみなす
        - なぜなら，chainが長ければ長いほど，多くの計算資源が注ぎ込まれたことになるから
        - 多数決ではなく，**one-CPU-one-vote**

In [37]:
from uuid import uuid4
from time import sleep
from copy import deepcopy
import multiprocessing as mp
import random
from itertools import count

- nodeを実装する
- networkまわりの細かい作業は，network interfaceに任せる

In [ ]:
difficulty=3
class Node:
    def __init__(self, network, genesis, uuid=None):
        self.chain=BlockChain([genesis])
        self.uuid=uuid or str(uuid4())
        self.network=network
        
    def work(self, verbose=False):
        while True:
            if not self.mine():
                sleep(0.1) # wait due to no transactions
            else:
                if verbose: print(self.uuid, "Mined a block")
            if self.add_block():
                if verbose: print(self.uuid, "Added one block")
            self.network.post_chain(self.chain, self.uuid) # publish my blockchain
            if self.resolve_conflicts():
                if verbose: print(self.uuid, "Change chain")
    
    def mine(self):
        transactions=self.network.get_transactions(self.uuid)
        if len(transactions)==0:
            return False # cannot mine due to no transactions
        previous_hash=self.chain[-1].hash()
        block=Block(time(), tuple(transactions), previous_hash)
        
        for i, n in enumerate(self.random_generator()):
            block.nonce=n
            if self.verify_proof(block):
                break
                
            if i % 100 == 0 and self.add_block(): # someone mined a block
                return True
            if i % 100 == 0 and self.resolve_conflicts(): # someone mined a block
                return True
                
        self.network.broadcast_block(block, self.uuid)
        return True # successfully mined
    
    @staticmethod
    def random_generator(step=None):
        step = step or random.randint(1e4, 1e5)
        for c in count():
            for i in random.sample(range(c*step, (c+1)*step), step):
                yield i
        
    def add_block(self):
        for block in self.network.get_blocks(self.uuid):
            if self.verify_block(block):
                self.chain.append(block)
                if self.verify_chain(self.chain):
                    return True 
                else:
                    self.chain.pop(-1)
        return False # no block is added
    
    def resolve_conflicts(self):
        """Longest valid chain is authoritative"""
        authoritative_chain=self.chain
        for chain in self.network.get_neighbour_chains(self.uuid):
            if not self.verify_chain(chain):
                # node is incorrect
                continue
            if len(chain)>len(authoritative_chain):
                # Longest valid chain is authoritative
                authoritative_chain=deepcopy(chain)
        self.chain=authoritative_chain
        return self.chain is not authoritative_chain
        
    @staticmethod
    def verify_transaction(transaction):
        if transaction.sign is None:
            return False
        h=SHA.new(transaction.str_data().encode())
        verifier=PKCS1_v1_5.new(encode_key(transaction.sender_address))
        return verifier.verify(h, binascii.unhexlify(transaction.sign))
    
    @staticmethod
    def verify_proof(block):
        return block.hash()[:difficulty] == "0" * difficulty
    
    def verify_block(self, block) -> bool:
        is_correct_transactions = all(map(self.verify_transaction, block.transactions))
        is_correct_proof = self.verify_proof(block)
        return is_correct_transactions and is_correct_proof
    
    def verify_chain(self, chain):
        for i in range(len(chain)-1, 0, -1):
            if not self.verify_block(chain[i]):
                return False
            if chain[i-1].hash() != chain[i].previous_hash:
                return False
        return True

- 全体的に，networkの作業が追加されている
- mineを少し変更している
    - 複数のnodeが同じ方法でマイニングをするのは無駄が生じる可能性がある
        - 例えば全く同じBlockのマイニングを複数のノードが開始した場合，確実に最も計算力の高いnodeがマイニングを成功させることになる

- Walletもネットワーク対応に
    - transactionを発行したら，知っているnodeにtransactionを送る

In [ ]:
class Wallet:
    def __init__(self, network, nodes=None):
        key=RSA.generate(1024)
        self.private_key = decode_key(key)
        self.address = decode_key(key.publickey())
        self.network=network
        self.nodes=nodes or []
    
    def sign_transaction(self, transaction):
        signer=PKCS1_v1_5.new(encode_key(self.private_key))
        h=SHA.new(transaction.str_data().encode())
        return dc.replace(transaction, sign=signer.sign(h).hex())
    
    def send(self, receiver_address, value):
        transaction=Transaction(self.address, receiver_address, value)
        self.broadcast(self.sign_transaction(transaction))
    
    def broadcast(self, transaction):
        for uuid in self.nodes:
            self.network.post_transaction(transaction, uuid)

- networkっぽいものを実装する
    - Network classを中継点としてデータの遣り取りをする
- あとでmultiprocessでノードを動かすために，データの管理はmultiprocess.managerで行う
- flaskとかで作っても面白いかもしれない
- 本質的でないので読み飛ばして良い

In [70]:
from typing import List

In [ ]:
class Network:
    def __init__(self, neighbours=()):
        self.manager=mp.Manager()
        self.chains=self.manager.dict()
        self.blocks=self.manager.dict()
        self.transactions=self.manager.dict()
        self.neighbours=dict(neighbours)
        
    def post_chain(self, chain, uuid):
        self.chains[uuid]=chain.json_dumps()
    
    def get_chain(self, uuid) -> BlockChain:
        if uuid not in self.chains:
            return []
        return BlockChain.json_loads(self.chains[uuid])
    
    def get_neighbour_chains(self, uuid) -> List[BlockChain]:
        return [self.get_chain(neigh) for neigh in self.neighbours[uuid] if neigh != uuid]
    
    def post_block(self, block, uuid):
        if uuid not in self.blocks:
            self.blocks[uuid]=self.manager.list([block.json_dumps()])
        else:
            self.blocks[uuid].append(block.json_dumps())
            
    def broadcast_block(self, block, uuid):
        for receiver in self.neighbours[uuid]:
            self.post_block(block, receiver)
            
    def get_blocks(self, uuid) -> List[Block]:
        if uuid not in self.blocks:
            return []
        res=[Block.json_loads(s) for s in self.blocks[uuid]]
        self.blocks[uuid][:]=[]
        return res
    
    def post_transaction(self, transaction, uuid):
        if uuid not in self.transactions:
            self.transactions[uuid]=self.manager.list([transaction.json_dumps()])
        else:
            self.transactions[uuid].append(transaction.json_dumps())
            
    def get_transactions(self, uuid) -> Tuple[Transaction]:
        if uuid not in self.transactions:
            return []
        res=[Transaction.json_loads(s) for s in self.transactions[uuid]]
        self.transactions[uuid][:]=[]
        return res
    
    def shutdown(self):
        self.manager.shutdown()

## ノード1つの場合のテスト

In [42]:
genesis=Block(time(), (), "0")
uuid="test"
network=Network({uuid:[uuid]})
node=Node(network, genesis, uuid)

In [43]:
alice=Wallet(network, [uuid])
bob=Wallet(network, [uuid])

In [44]:
alice.send(bob.address, 5)

- これを実行してしばらく問題が起きなければOK
    - 適当にinterruptする

In [45]:
# node.work(verbose=True)

- 後片付け

In [46]:
network.shutdown()

# デモ: 分散blockchainのデモ
- multiprocessingでnetworkのデモを行う

In [47]:
genesis=Block(time(), (), "0")
network=Network()

- node生成
- 適当にnum_nodesを変更して遊ぶと良い

In [48]:
num_nodes=7
nodes=[str(uuid4()) for _ in range(num_nodes)]

- nodeのnetworkの状態を定義する
- とりあえずは全結合にする
    - つまり全てのNode同士，直接やりとりできる

In [49]:
for uuid in nodes:
    network.neighbours[uuid]=nodes

- multiprocess workerを作って起動する

In [50]:
def node_work(uuid):
    node=Node(network, genesis, uuid)
    node.work()

In [51]:
workers=[]
for uuid in nodes:
    worker=mp.Process(target=node_work, args=(uuid,))
    workers.append(worker)
    worker.start()

- peopleを生成する
- これも適当に人数を変えて遊ぶと良い

In [52]:
num_people=5
people=[Wallet(network, random.sample(nodes, random.randint(1, len(nodes)))) for _ in range(num_people)]

### blockchain の表示
- 各ノードの持つblockchainがどの様になっているのか表示する
- 別Processで行う
    - Jupyterは賢いのでoutputにきちんと表示してくれる

In [53]:
from IPython.display import clear_output
def draw_chains(network):
    while True:
        line=""
        for k, v in network.chains.items():
            line += f"(Node: {k[:4]}) "
            chain=BlockChain.json_loads(v)
            h=list(map(lambda x: x.hash()[:5], chain))
            line += "--".join(map(lambda x: f"[{x}]", h))
            line += "\n"
        print(line.rstrip())
        clear_output(True)
        sleep(0.1)

worker=mp.Process(target=draw_chains, args=(network,))
worker.start()

(Node: 7301) [9e462]--[00001]
(Node: 0aab) [9e462]--[00001]
(Node: 76a6) [9e462]--[00001]
(Node: 381e) [9e462]--[00001]
(Node: 0887) [9e462]
(Node: 6207) [9e462]--[00001]
(Node: 0443) [9e462]


- 取引をして遊ぶ
    - ランダムに取引が行われるコードを書いてみる
    - 上の表示が変化する!

In [54]:
def generate_transactions():
    for _ in range(random.randint(1,10)):
        sender, receiver= random.sample(people, 2)
        sender.send(receiver.address, random.randint(1, 100))

In [55]:
generate_transactions()

- networkをちょと変えて遊んで見る
    - ちょっと断線させる

In [56]:
for uuid in nodes:
    network.neighbours[uuid]=random.sample(nodes, 3)

In [57]:
generate_transactions()

### 片付け

In [58]:
worker.terminate()
worker.kill()
for worker in workers:
    worker.terminate()
    worker.kill()
network.shutdown()

# おわりに
- 本物のBitCoinとは異なる部分がたくさんあります

# 参考文献
- [Bitcoin white paper](https://bitcoin.org/bitcoin.pdf)
    - 専門外でも読みやすいです
- [Bitcoin Wiki](https://en.bitcoin.it/wiki/Main_Page)
    - 非常に情報量が多いです